# **Retail Revenue Prediction - Short Notebook for Kaggle Submission:** *pred_new_stripped_150outliers_3feat.csv*

##### **Group Information**
**Participants:**
- Mina Chen Glein Feragen - 544552
- Andrew Glover Marty - 557813
- Simen Tvete Aabol - 505174


**Kaggle Team name:** Group 8

## Short about the prediction

For this prediction we used gradient boosting, as we found that the algorithm gave accurate predictions and at the same time being easy to implement.

As for the features, we first used all of the features we had computed at the time, and then plotted the importance of each feature, as can be seen in the long notebook. Based on these foundings, we chose to "strip" away 'lv1', 'busstop_importance_level', 'municipality_name', 'distance_from_busstop', 'district_municipality_name', since these were the least important according to the algorithm itself. We did however also make predictions with all of the features, which can be seen commented out throughout this notebook, but we chose to go with the "stripped" version, as it had performed better with other combinations as well, and we wanted to avoid overfitting. 

In [ ]:
import pandas as pd
import numpy as np
import math

from sklearn.preprocessing import LabelEncoder
import geopandas as gpd
from shapely.geometry import Point
import re
import copy

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

NameError: name 'pd' is not defined

## Data Preprocessing and Feature Engineering

In [ ]:
"""
To prepare for the feature engineering, we replaced categorical values in the busstops-table with numerical ones,
and made own columns for latitude and longditude, so that computing the distance to a busstop would be 
easier in the future.
"""

busstops = pd.read_csv('./data/busstops_norway.csv')
busstops = busstops.drop(columns=["stopplace_type", "side_placement"])

importance_levels = ["Mangler viktighetsnivå", "Standard holdeplass", "Lokalt knutepunkt",
                     "Regionalt knutepunkt", "Annen viktig holdeplass", "Nasjonalt knutepunkt"]
numerated_importance_levels = [1, 2, 4, 5, 3, 6]

busstops["importance_level"] = busstops["importance_level"].replace(importance_levels, numerated_importance_levels)

busstops.insert(3, "lat", -math.inf)
busstops.insert(4, "lon", -math.inf)

busstops_array = []
for row_index in range(len(busstops)):
    coordinates = busstops["geometry"][row_index][6:-2].split(' ')
    busstops["lon"][row_index] = float(coordinates[0])
    busstops["lat"][row_index] = float(coordinates[1])
    busstops_array.append([float(coordinates[1]), float(coordinates[0])])
busstops = busstops.drop(columns="geometry")


In [3]:
# We also wanted to add other data from the given csv-files, which is added to the stores_train-DataFrame by this function

def addStoreData(df):
    df.pop("sales_channel_name")
    df.pop("address")
    df.pop("year")
    df.pop("store_name")
    
    try:
        try_accessing = df["revenue"]
        df = copy.deepcopy(df[df.revenue != 0])
        df.at[7435, 'chain_name'] = np.nan 
        df.at[8164, 'chain_name'] = "KIWI NORGE"
    except KeyError:
        print(" Revenue doesn't exist in dataframe df")
 
    plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')
    plaace_hierarchy.pop('lv1_desc')
    plaace_hierarchy.pop('lv2_desc')
    plaace_hierarchy.pop('lv3_desc')
    plaace_hierarchy.pop('lv4_desc')
    plaace_hierarchy.pop('sales_channel_name')
    df = pd.merge(df, plaace_hierarchy, how='left', on='plaace_hierarchy_id')
    df.pop('plaace_hierarchy_id') 

    for index, row in df.iterrows():
        df.at[index, 'lv2'] = int(str(df.loc[index, 'lv2']).replace('.', ''))
        df.at[index, 'lv3'] = int(df.loc[index, 'lv3'].replace('.', ''))
        df.at[index, 'lv4'] = int(df.loc[index, 'lv4'].replace('.', ''))

    
    districts = pd.read_csv('data/grunnkrets_norway_stripped.csv')
    districts.pop("year")
    districts.pop('grunnkrets_name')
    districts.pop("area_km2")
    districts = districts.drop_duplicates()
    districts = districts.reset_index(drop=True)

    
    
    
    grunnkrets_household_income = pd.read_csv('data/grunnkrets_income_households.csv')
    grunnkrets_household_income.head()
    grunnkrets_household_income.pop("singles")
    grunnkrets_household_income.pop("couple_without_children")
    grunnkrets_household_income.pop("couple_with_children")
    grunnkrets_household_income.pop("other_households")
    grunnkrets_household_income.pop("single_parent_with_children")
    grunnkrets_household_income = grunnkrets_household_income[grunnkrets_household_income.year != 2015]
    grunnkrets_household_income.pop("year")
    
    income_districts = pd.merge(grunnkrets_household_income,districts, how = 'right', on = 'grunnkrets_id')
    income_districts.rename(columns = {'all_households':'households_grunnkrets'}, inplace = True)
    
    df = pd.merge(df,income_districts, how = 'left', on = 'grunnkrets_id')
    

    # Mall name
    df['mall_name'] = df['mall_name'].replace(np.nan, 0)
    for i in range(len(df) ):
        if( df['mall_name'][i] != 0):
            df.loc[i,'mall_name'] = 1
    df['mall_name'] = pd.to_numeric(df['mall_name'])
            
            
         
    # Chain name  
    df['chain_name'] = df['chain_name'].replace(np.nan, "A-not a chain") 
    threshold = 6 # Anything that occurs less than this will be removed.
    for col in df.columns:
        value_counts = df['chain_name'].value_counts() # Specific column 
        to_remove = value_counts[value_counts <= threshold].index
        df['chain_name'].replace(to_remove, "A-not a chain", inplace=True)
        
    df['chain_name'] = LabelEncoder().fit_transform(df['chain_name'])
    
    
        
        
    # code that handle missing data
        
    df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
    store_gdf = gpd.GeoDataFrame(df, geometry='geometry')
    store_gdf = store_gdf.drop_duplicates()


    for index, row in df.iterrows():
        pattern = r'[0-9]'
        
        if pd.isnull(row['municipality_name']):
            lat = row['lat']
            lon = row['lon']
            
            store_location = Point(lon, lat)
        
            polygon_indices_municipality_name = store_gdf.distance(store_location).sort_values().index[0:1]
            nearest_grunnkretser = store_gdf.loc[polygon_indices_municipality_name]

                    
            pair_value = nearest_grunnkretser['municipality_name'].sort_values()
            one_pair = pair_value[0:1]
            str_pair = str(one_pair)
            str_pair = re.sub(pattern, '', str_pair)        
            word = str_pair.split()[0]
            df.at[index, 'municipality_name'] = word
        
        if pd.isnull(row['district_name']):
            lat = row['lat']
            lon = row['lon']
            
            store_location = Point(lon, lat)
        
            polygon_indices_district_name = store_gdf.distance(store_location).sort_values().index[0:1]
            nearest_grunnkretser = store_gdf.loc[polygon_indices_district_name]
            
            pair_value = nearest_grunnkretser['district_name'].sort_values()
            one_pair = pair_value[0:1]
            str_pair = str(one_pair)
            str_pair = re.sub(pattern, '', str_pair)        
            word = str_pair.split()[0]
            df.loc[index, 'district_name'] = word
            
        if pd.isnull(row['households_grunnkrets']):
            lat = row['lat']
            lon = row['lon']
            
            store_location = Point(lon, lat)
        
            polygon_indices_all_households = store_gdf.distance(store_location).sort_values().index[0:4]
            nearest_grunnkretser = store_gdf.loc[polygon_indices_all_households]
            
            df.loc[index, 'households_grunnkrets'] = nearest_grunnkretser['households_grunnkrets'].mean()
            

    

    df['district_municipality_name'] = df['district_name'] + '-'+ df['municipality_name']
    df['district_municipality_name'] = LabelEncoder().fit_transform(df['district_municipality_name'])
    df['municipality_name'] = LabelEncoder().fit_transform(df['municipality_name'])
    

    df.pop('geometry')

    return df

In [ ]:
"""
Back to the busstops, we wanted to compute the distance to the closest busstop, and also what we called "weighted bus data", which computes
the product of the busstop importance level and the distance, and takes the 10-log of that, which we found to be the most effective.
"""

busstops_array = []
for row_index in range(len(busstops)):
    busstops_array.append([busstops["lat"][row_index], busstops["lon"][row_index]])
    
# Defining a function that finds the closest busstop and the distance
def getDistFromBusStop(store_lat, store_lon):
        busstops_big_array = np.asarray(busstops_array)
        distances = np.sum((busstops_big_array-[store_lat, store_lon])**2, axis=1)
        index_busstop = np.argmin(distances)
        shortest_distance = distances[index_busstop]
        return shortest_distance, busstops.at[index_busstop, "importance_level"]

def addWeightedBusData(df):
    for index, row in df.iterrows():
         # Adding distance to closest busstop
        distance, importance_level = getDistFromBusStop(row['lat'], row['lon'])
        df.at[index, "distance_from_busstop"] = distance
        df.at[index, "busstop_importance_level"]= importance_level
        
    df.insert(11, 'buss', -math.inf)
    df['buss'] = np.log10(np.multiply(df['distance_from_busstop'], df['busstop_importance_level']))
    
    return df
    
    

In [ ]:
def addPopulation(stores_data):
    """ Returns population of grunnkrets and district
        Manipulate or remove last line of code according to your taste :)
        May require some more work for imputation of 'grunnkrets_population' using data from 'stores_extra' using 'lat' and 'lon'
        Possibility of using unspervised learning?
        Can be used for training only.Need to see how to make it multipurpose for test data as well.
        Else Separate function is needed.
    """
    input_data = copy.deepcopy(stores_data)
    grunnkrets_ages = pd.read_csv('data/grunnkrets_age_distribution.csv')
    grunnkrets = pd.read_csv('data/grunnkrets_norway_stripped.csv')
    #Get data for every grunnkrets_id and and drop duplicates. Prioritize the year '2016'
    grunnkrets = grunnkrets.sort_values('year', ascending=False).drop_duplicates('grunnkrets_id').sort_index()
    
    #Create District+Municipality
    grunnkrets_merged_district_municipality_name = grunnkrets
    grunnkrets_merged_district_municipality_name['district_name_pro'] = grunnkrets['district_name'] + ' '+ grunnkrets['municipality_name']
    
    #Drop columns except 'grunnkrets_id' and 'district_municipality_name'
    grunnkrets_with_district_municipality_names = grunnkrets_merged_district_municipality_name
    grunnkrets_with_district_municipality_names = grunnkrets_with_district_municipality_names.drop(grunnkrets_with_district_municipality_names.iloc[:,1:5].columns,axis =1)
    
    #Get data for every grunnkrets_id and and drop duplicates. Prioritize the year '2016'
    grunnkrets_ages_new = grunnkrets_ages.sort_values('year', ascending=False).drop_duplicates('grunnkrets_id').sort_index()
    
    #Sum all ages in grunnkrets
    grunnkrets_ages_new['population'] = grunnkrets_ages_new.iloc[:,2:].sum(axis =1)
    
    #Clean: Drop all age columns including year column
    grunnkrets_population = grunnkrets_ages_new
    grunnkrets_population = grunnkrets_population.drop(grunnkrets_population.iloc[:,1:93].columns,axis =1)
    
    grunnkrets_population_dist_muni = grunnkrets_population
    grunnkrets_population_dist_muni = pd.merge(grunnkrets_with_district_municipality_names,grunnkrets_population, how = 'left', on = 'grunnkrets_id')
    
    grunnkrets_dist_muni = grunnkrets_population_dist_muni
    grunnkrets_dist_muni = grunnkrets_dist_muni.drop(['population'], axis = 1)
    
    #Merge only grunnkrets population and find missing population before merging
    merge_grunnkrets_populn_stores_train = pd.merge(stores_data, grunnkrets_dist_muni, how = 'left', on = 'grunnkrets_id')
    
    #Add population of district
    st_train_grunn_pp = pd.merge(merge_grunnkrets_populn_stores_train,grunnkrets_population, how = 'left', on = 'grunnkrets_id')
    
    #Get Population of District
    population_dist_muni = st_train_grunn_pp
    population_dist_muni = population_dist_muni.groupby('district_name_pro')['population'].sum()
    
    #Merge
    st_train_grunn_pp_dist_pp = pd.merge(st_train_grunn_pp,population_dist_muni, how = 'left', on = 'district_name_pro')
    
    st_train_grunn_pp_dist_pp.rename(columns = {'population_x':'grunnkrets_population','population_y': 'district_population'}, inplace = True)
    
    st_train_grunn_pp_dist_pp['geometry'] = gpd.GeoSeries.from_wkt(st_train_grunn_pp_dist_pp['geometry'])

    store_gdf = gpd.GeoDataFrame(st_train_grunn_pp_dist_pp, geometry='geometry')
    store_gdf = store_gdf.drop_duplicates()
    
    for index, row in st_train_grunn_pp_dist_pp.iterrows():
        # print("row", row)
    
        if pd.isnull(row['grunnkrets_population']):
            lat = row['lat']
            lon = row['lon']
            
            store_location = Point(lon, lat)
        
            polygon_indices = store_gdf.distance(store_location).sort_values().index[0:150] #lower values returns missing data for grunnkrets_population
            #cannot guarantee accuracy of imputed missing population
            nearest_grunnkretser = store_gdf.loc[polygon_indices]
            
            st_train_grunn_pp_dist_pp['grunnkrets_population'].loc[index] = np.floor(nearest_grunnkretser['grunnkrets_population'].mean())
        
    for index, row in st_train_grunn_pp_dist_pp.iterrows():        
        if pd.isnull(row['district_population']):
            lat = row['lat']
            lon = row['lon']
            
            store_location = Point(lon, lat)
        
            polygon_indices = store_gdf.distance(store_location).sort_values().index[0:4]
            nearest_grunnkretser = store_gdf.loc[polygon_indices]
            
            st_train_grunn_pp_dist_pp['district_population'].loc[index] = np.floor(nearest_grunnkretser['district_population'].mean())
    
    st_train_grunn_pp_dist_pp = st_train_grunn_pp_dist_pp.drop(st_train_grunn_pp_dist_pp.iloc[:,1:14].columns,axis =1)#Take off this if all colmuns are needed
    
    return pd.merge(input_data, st_train_grunn_pp_dist_pp, how="left", on="store_id").drop_duplicates('store_id')


In [ ]:
# We thought some revenues would not have a representative revenue, and chose therefore to remove the top and bottom 150, which is ~2.5% of the data
def removeNOutliers(df, n = 150):
    df = copy.deepcopy(df[df['revenue'] != 0])
    df = df.sort_values(by='revenue')
    return df[n:-n]

In [ ]:
# To preprocess the data we made a common function for all features and other data preprocessing
def preprocess_data(input_data):
    
    input_data = addPopulation(input_data)
    input_data = addStoreData(input_data)
    input_data = addWeightedBusData(input_data)
    
    training_data = removeNOutliers(copy.deepcopy(input_data[input_data['revenue']!=-math.inf]))
    test_data = copy.deepcopy(input_data[input_data['revenue']==-math.inf])
    
    input_data = training_data.append(test_data)
    
    input_data['district_municipality_name'] = LabelEncoder().fit_transform(input_data['district_municipality_name'])
    input_data['municipality_name'] = LabelEncoder().fit_transform(input_data['municipality_name'])
    input_data['mall_name'].replace(np.nan(), 0)
    
    input_data = input_data.drop(columns=[])
    return input_data

In [ ]:
stores_train = pd.read_csv("./data/stores_train.csv")
stores_extra = pd.read_csv("./data/stores_extra.csv")
stores_extended = stores_train.append(stores_extra)

In [ ]:
stores_test = pd.read_csv('data/stores_test.csv')
stores_test.insert(len(stores_test.columns), column="revenue", value=-math.inf)

In [ ]:
all_stores = pd.DataFrame(stores_extended).append(stores_test)

In [ ]:
preprocessed = preprocess_data(all_stores)

In [ ]:
stores_train_preprocessed = copy.deepcopy(preprocessed[preprocessed['revenue']!=-math.inf])
stores_test_preprocessed = copy.deepcopy(preprocessed[preprocessed['revenue']==-math.inf])

In [ ]:
for index, row in stores_test_preprocessed.iterrows(): # Finding out levels for the stores who missed it
    if (pd.isna(stores_test_preprocessed.loc[index, 'lv4'])):
        lv4 = stores_test_preprocessed.loc[index, 'plaace_hierarchy_id'].replace('.', '')
        stores_test_preprocessed.at[index, 'lv1'] = int(lv4[0])
        stores_test_preprocessed.at[index, 'lv2'] = int(lv4[0:2])
        stores_test_preprocessed.at[index, 'lv3'] = int(lv4[0:3])
        stores_test_preprocessed.at[index, 'lv4'] = int(lv4)

In [ ]:
# Dropping categorical columns
stores_train_preprocessed = stores_train_preprocessed.drop(columns=['store_name', 'plaace_hierarchy_id','sales_channel_name', 'address', 'year'])
stores_test_preprocessed = stores_test_preprocessed.drop(columns=['store_name', 'plaace_hierarchy_id','sales_channel_name', 'address', 'year'])

In [ ]:
# Removing NaN-data and transforming ID
for row in range(0, len(stores_train_preprocessed)):
    stores_train_preprocessed.iloc[row, 0] = int(stores_train_preprocessed.iloc[row, 0].replace('-', ''))

stores_train_preprocessed = stores_train_preprocessed[stores_train_preprocessed['district_population'].notna()]
stores_train_preprocessed = stores_train_preprocessed[stores_train_preprocessed['grunnkrets_population'].notna()]
stores_train_preprocessed = copy.deepcopy(stores_train_preprocessed[stores_train_preprocessed['households_grunnkrets'].notnull()])
stores_train_preprocessed = copy.deepcopy(stores_train_preprocessed[stores_train_preprocessed['revenue'].notna()])

In [ ]:
X_stripped = pd.DataFrame(stores_train_preprocessed).drop(columns=['revenue', 'lv1', 'busstop_importance_level', 'municipality_name', 'distance_from_busstop', 'district_municipality_name'])
""" X_all = pd.DataFrame(stores_train_preprocessed.drop(columns='revenue')) """
y = stores_train_preprocessed['revenue']
y_log = np.log10(y*10000) #Multiplying by 10 000 to avoid negative target values

### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

#### Defining and testing base model

In [ ]:
for depth in range(1, 10):
    """ tree_regressor_all = tree.DecisionTreeRegressor(max_depth=depth, random_state=1)
    if tree_regressor_all.fit(X_all, y_log).tree_.max_depth < depth:
        break
    score_all=np.mean(cross_val_score(tree_regressor_all, X_all, y_log,
                                  scoring = 'neg_mean_squared_log_error'))
    score_all=math.sqrt(abs(score_all))
    print('All: ', depth, score_all) """
    
    tree_regressor_stripped = tree.DecisionTreeRegressor(max_depth=depth, random_state=1)
    if tree_regressor_stripped.fit(X_stripped, y_log).tree_.max_depth < depth:
        break
    score_stripped=np.mean(cross_val_score(tree_regressor_stripped, X_stripped, y_log,
                                  scoring = 'neg_mean_squared_log_error'))
    score_stripped=math.sqrt(abs(score_stripped))
    print('Stripped: ', depth, score_stripped) #Checking how well the base model was performing

#### Hyperparameter Tuning

In [ ]:
GBR=GradientBoostingRegressor()
search_grid = {'n_estimators':[50, 100, 200], 'learning_rate': [0.001, 0.01, 0.1],
               'max_depth': [1, 2, 4], 'subsample': [0.5, 0.75, 1], 'random_state': [1]}
search_all=GridSearchCV(estimator=GBR, param_grid=search_grid,
                    scoring='neg_mean_squared_log_error')
search_stripped=GridSearchCV(estimator=GBR, param_grid=search_grid,
                    scoring='neg_mean_squared_log_error')

# Using GridSearch to find the best hyperparameters

In [ ]:
""" search_all.fit(X_all, y_log)
print(search_all.best_params_)
score_all = math.sqrt(abs(search_all.best_score_))
print("All: ", score_all) """

search_stripped.fit(X_stripped, y_log)
print(search_stripped.best_params_)
score_stripped = math.sqrt(abs(search_stripped.best_score_))
print("Stripped", score_stripped)

#### Creating actual models

In [ ]:
""" # Best values from GridSearchCV
GBR2_all = GradientBoostingRegressor(n_estimators=search_all.best_params_['n_estimators'], learning_rate=search_all.best_params_['learning_rate'],
                                 subsample=search_all.best_params_['sub_sample'],max_depth=search_all.best_params_['max_depth'], random_state=1)
score_all=np.mean(cross_val_score(GBR2_all, X_all, y_log, scoring='neg_mean_squared_log_error', n_jobs=1))
score_all=math.sqrt(abs(score_all)) """

# Best values from GridSearchCV
GBR2_stripped = GradientBoostingRegressor(n_estimators=search_stripped.best_params_['n_estimators'], learning_rate=search_stripped.best_params_['learning_rate'],
                                 subsample=search_stripped.best_params_['sub_sample'],max_depth=search_stripped.best_params_['max_depth'], random_state=1)
score_stripped=np.mean(cross_val_score(GBR2_stripped, X_stripped, y_log, scoring='neg_mean_squared_log_error', n_jobs=1))
score_stripped=math.sqrt(abs(score_stripped))

In [ ]:
""" GBR2_all.fit(X_all, y_log) """
GBR2_stripped.fit(X_stripped, y_log)

#### Making predictions

In [ ]:
stores_test_ids = copy.deepcopy(stores_test_preprocessed['store_id']) #Saving IDs in the right format for submitting to Kaggle

for row in range(0, len(stores_test_preprocessed)):
    stores_test_preprocessed.iloc[row, 0] = int(stores_test_preprocessed.iloc[row, 0].replace('-', ''))

stores_test_preprocessed['households_grunnkrets'] = stores_test_preprocessed['households_grunnkrets'].replace(np.nan, stores_test_preprocessed['households_grunnkrets'].mean())
stores_test_preprocessed['grunnkrets_population'] = stores_test_preprocessed['grunnkrets_population'].replace(np.nan, stores_test_preprocessed['grunnkrets_population'].mean())
stores_test_preprocessed['district_population'] = stores_test_preprocessed['district_population'].replace(np.nan, stores_test_preprocessed['district_population'].mean())

In [ ]:
""" prediction_all = (10**(GBR2_all.predict(stores_test_preprocessed.drop(columns='revenue'))))/10000 """
prediction_stripped = (10**(GBR2_stripped.predict(stores_test_preprocessed.drop(columns=['revenue', 'lv1', 'municipality_name', 'district_municipality_name', 'distance_from_busstop', 'busstop_importance_level']))))/10000

""" data_all = {"id": stores_test_ids, 
        "predicted": prediction_all} """

data_stripped = {"id": stores_test_ids, 
        "predicted": prediction_stripped}
""" 
prediction_all_submission = pd.DataFrame(data_all) """
prediction_stripped_submission = pd.DataFrame(data_stripped)
""" 
prediction_all_submission.to_csv("./predictions/pred_all_150outliers_3feat.csv", index=False) """
prediction_stripped_submission.to_csv("./predictions/pred_new_stripped_150outliers_3feat.csv", index=False)